In [ ]:
import random

import numpy as np
import matplotlib.pyplot as plt

import shapely.geometry as geo
import shapely.affinity as aff
from descartes.patch import PolygonPatch

In [ ]:
random.seed(1618)

In [ ]:
colours = plt.cm.viridis(np.linspace(0,1,1000))
golden_ratio = (1 + np.sqrt(5)) / 2

In [ ]:
bound = 4
body_buffer = 0.14
head_buffer = body_buffer * golden_ratio
head_stretch = 1.3

expected_x_bounds = np.array([-bound - body_buffer, bound + head_buffer])
length = np.diff(expected_x_bounds)
height = length / golden_ratio
desired_amplitude = height / 2 - body_buffer

def snake_wave_packet(x):
    x = np.array(x)
    return desired_amplitude * np.exp(-x**2/2) * np.sin(2 * np.pi * x) 

expected_y_bounds = np.ravel([-height/2, height/2])

In [ ]:
epsilon = 0.1
x_init = bound - head_buffer * head_stretch

y_vals = snake_wave_packet([x_init, x_init + epsilon])
ydiff = np.diff(y_vals)
y_init = y_vals[0]

head_shift = head_buffer * head_stretch
hypot = np.sqrt(epsilon**2 + ydiff**2)
num_shifts = head_shift / hypot
neck_shring = 1
# assert np.diff(expected_x_bounds) / np.diff(expected_y_bounds) == golden_ratio

head_angle = np.arctan(ydiff / epsilon) / np.pi * 180
head_middle_x = x_init + epsilon * num_shifts * neck_shring
head_middle_y = y_init + ydiff * num_shifts * neck_shring

In [ ]:
tongue_length = 0.1
tongue_buffer = 0.02
fork_angles = 35

tongue_init_x = head_middle_x + epsilon * num_shifts
tongue_init_y = head_middle_y + ydiff * num_shifts

tongue_shift_num = tongue_length / hypot

tongue_shift_x = epsilon * tongue_shift_num
tongue_shift_y = ydiff * tongue_shift_num

tongue_fork_x = tongue_init_x + tongue_shift_x
tongue_fork_y = tongue_init_y + tongue_shift_y

tongue_base = geo.LineString(
    [(tongue_init_x, tongue_init_y), (tongue_fork_x, tongue_fork_y)]
).buffer(tongue_buffer)

tongue_base

tongue_top_fork = aff.rotate(tongue_base, fork_angles, origin=(tongue_init_x, tongue_init_y))
tongue_top_fork = aff.translate(tongue_top_fork, tongue_shift_x, tongue_shift_y)

tongue_bot_fork = aff.rotate(tongue_base, -fork_angles, origin=(tongue_init_x, tongue_init_y))
tongue_bot_fork = aff.translate(tongue_bot_fork, tongue_shift_x, tongue_shift_y)


tongue = tongue_base.union(tongue_top_fork).union(tongue_bot_fork)
tongue

In [ ]:
point = geo.Point(head_middle_x, head_middle_y)
head = point.buffer(head_buffer)
head = aff.scale(head, head_stretch, 1, origin=point)
head = aff.rotate(head, head_angle, origin=point)
head = head.union(tongue)

head

In [ ]:
x = np.linspace(-bound, x_init, 1000)
y = snake_wave_packet(x)
centre_line = geo.LineString(zip(x, y))
centre_line

In [ ]:
body = centre_line.buffer(body_buffer)
body

In [ ]:
snake = body.union(head)
snake

In [ ]:
xmin, ymin, xmax, ymax = snake.bounds

In [ ]:
ring_thickness = 2
bound_diff = 0.1
ring_outside = bound - bound_diff
ring_inside = ring_outside - ring_thickness

ring = geo.Point(0,0).buffer(ring_outside).difference(geo.Point(0,0).buffer(ring_inside))
ring

In [ ]:
def make_logo(snake_colour, ring_colour):
    fig = plt.figure(1, figsize=(10,10), dpi=90)
    fig.set_frameon(True)
    ax = fig.add_subplot(111)


    patch_ring = PolygonPatch(ring, facecolor=ring_colour)
    ax.add_patch(patch_ring)

    patch_snake = PolygonPatch(snake, facecolor=snake_colour)
    ax.add_patch(patch_snake)

    plt.xlim(-bound-1, bound+1)
    plt.ylim(-bound-1, bound+1)

    plt.show()
    

make_logo(colours[600], colours[400])

In [ ]:
make_logo(colours[600], colours[300])

In [ ]:
make_logo(colours[700], colours[500])

In [ ]:
make_logo(colours[800], colours[500])

In [ ]:
make_logo(colours[650], colours[400])

In [ ]:
make_logo(colours[700], colours[400])

In [ ]:
make_logo(colours[500], colours[400])

In [ ]:
make_logo(colours[630], colours[400])

In [ ]:
def make_logo(snake_colour, ring_colour):
    fig = plt.figure(1, figsize=(10,10), dpi=90)
    fig.set_frameon(True)
    ax = fig.add_subplot(111)


    patch_ring = PolygonPatch(ring, facecolor=ring_colour)
    ax.add_patch(patch_ring)

    patch_snake = PolygonPatch(snake, facecolor=snake_colour, linewidth=golden_ratio)
    ax.add_patch(patch_snake)

    plt.xlim(-bound-1, bound+1)
    plt.ylim(-bound-1, bound+1)
    
    plt.axis('off')
    plt.gca().set_position([0, 0, 1, 1])
    plt.savefig("logo.svg", transparent=True)

    plt.show()
    

make_logo(colours[630], colours[400])